[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/multimodal.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/laminlabs/lamin-usecases/main?labpath=lamin-usecases%2Fdocs%2Fmultimodal.ipynb)

# Multi-modal

scRNA data has moved beyond just RNA and can also include the measurements of other modalities such as chromatin accessibility, surface proteins or adaptive immune receptors.
ECCITE-seq is designed to enable interrogation of single-cell transcriptomes together with surface protein markers in the context of CRISPR screens.

Here, we'll showcase how to curate and register ECCITE-seq data from [Papalexi21](https://www.nature.com/articles/s41592-019-0392-0) in the form of [MuData](https://github.com/scverse/mudata) objects.

## Setup

In [ ]:
!lamin init --storage ./test-multimodal --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

lb.settings.species = "human"

In [ ]:
ln.track()

## Papalexi21

Let's use a MuData object:

### Transform ![](https://img.shields.io/badge/Transform-10b981)

In [ ]:
mdata = ln.dev.datasets.mudata_papalexi21_subset()
mdata

MuData objects build on top of AnnData objects to store and serialize multimodal data.
More information can be found on the [MuData documentation](https://mudata.readthedocs.io/en/latest/).

First we register the file:

In [ ]:
file = ln.File(
    "papalexi21_subset.h5mu", description="Sub-sampled MuData from Papalexi21"
)
file.save()

Now let's validate and register the 3 feature sets this data contains:
1. RNA (gene expression)
2. ADT (antibody derived tags reflecting surface proteins)
3. obs (metadata)

For the two modalities rna and adt, we use bionty tables as the reference:

### Validate ![](https://img.shields.io/badge/Validate-10b981)

In [ ]:
mdata["rna"].var_names[:5]

In [ ]:
lb.Gene.validate(mdata["rna"].var_names, lb.Gene.symbol);

In [ ]:
genes = lb.Gene.from_values(mdata["rna"].var_names, lb.Gene.symbol)
ln.save(genes)

In [ ]:
mdata["rna"].var_names = lb.Gene.standardize(mdata["rna"].var_names, lb.Gene.symbol)

In [ ]:
validated = lb.Gene.validate(mdata["rna"].var_names, lb.Gene.symbol)

In [ ]:
new_genes = [lb.Gene(symbol=symbol) for symbol in mdata["rna"].var_names[~validated]]
ln.save(new_genes)

In [ ]:
lb.Gene.validate(mdata["rna"].var_names, lb.Gene.symbol);

In [ ]:
feature_set_rna = ln.FeatureSet.from_values(
    mdata["rna"].var_names, field=lb.Gene.symbol
)

In [ ]:
mdata["adt"].var_names

In [ ]:
lb.CellMarker.validate(mdata["adt"].var_names);

In [ ]:
markers = lb.CellMarker.from_values(mdata["adt"].var_names)
ln.save(markers)

In [ ]:
lb.CellMarker.validate(mdata["adt"].var_names);

### Register ![](https://img.shields.io/badge/Register-10b981) 

In [ ]:
feature_set_adt = ln.FeatureSet.from_values(
    mdata["adt"].var_names, field=lb.CellMarker.name
)

Link them to file:

In [ ]:
file.features.add_feature_set(feature_set_rna, slot="rna")
file.features.add_feature_set(feature_set_adt, slot="adt")

The 3rd feature set is the obs:

In [ ]:
obs = mdata["rna"].obs

We're only interested in a single metadata column:

In [ ]:
ln.Feature(name="gene_target", type="category").save()

In [ ]:
features = ln.Feature.from_df(obs)
ln.save(features)

In [ ]:
feature_set_obs = ln.FeatureSet.from_df(obs)

In [ ]:
file.features.add_feature_set(feature_set_obs, slot="obs")

In [ ]:
gene_targets = lb.Gene.from_values(obs["gene_target"], lb.Gene.symbol)
ln.save(gene_targets)
features = ln.Feature.lookup()
file.labels.add(gene_targets, feature=features.gene_target)

In [ ]:
nt = ln.ULabel(name="NT", description="Non-targeting control of perturbations")
nt.save()

In [ ]:
file.labels.add(nt, feature=features.gene_target)

In [ ]:
for col in ["orig.ident", "perturbation", "replicate", "Phase", "guide_ID"]:
    labels = [ln.ULabel(name=name) for name in obs[col].unique()]
    ln.save(labels)

Because none of these labels seem like something we'd want to track in the registry or validate, we don't link them to the file.

In [ ]:
file.features

In [ ]:
file.describe()

In [ ]:
file.view_flow()

In [ ]:
# clean up test instance
!lamin delete --force test-multimodal
!rm -r test-multimodal